# Generate a Dynamic Graph
In this notebook, we generate a dynamic graph based on a synthetic LFR graph, where we define the tranformation, the speed and the parameters of the initial graph.
If your goal is to generate multiple graph copies for a statistical comparison of algorithms, consider instead the usage of `../experiments/01a_generate_graphs.py`.

在这个笔记本中，我们基于合成的 LFR 图生成动态图，在其中定义初始图的变换、速度和参数。如果你的目标是生成多个图副本以对算法进行统计比较，那么不妨考虑使用 ../experiments/01a_generate_graphs.py 。

In [1]:
import sys
print(sys.executable)

d:\python\Anaconda\python.exe


In [3]:
import sys
sys.path.append('../')
from src.simulation import GraphMovementSimulation
import networkx as nx
from tqdm import tqdm
import os
import pandas as pd

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap', 'leidenalg', 'graph_tool', 'bayanpy'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'leidenalg', 'infomap'}


Chose a transformation among those in the list:
这个列表是社区动态演化中可用的社区变换操作，通常用于模拟动态网络中社区结构随时间的变化。每个变换操作对应一种特定的社区演化行为 
- 'birth',                  （新生，在图中新增一个社区）
- 'fragment',               （碎片化，将一个社区分裂成多个较小的子社区，但不会完全消失）
- 'split',                  （分裂，将一个社区完全分裂成两个或多个独立的新社区）
- 'merge',                  （合并，将两个或者多个社区合并成一个更大的社区）
- 'add_nodes',              （新增节点，向图中添加新节点，并可能分配到现有社区或形成新社区）
- 'intermittence_nodes',    （节点间歇性出现，某些节点在特定时间步出现或消失）
- 'switch',                 （切换，节点从一个社区切换到另一个社区）
- 'break_up_communities',   （社区瓦解，完全解散一个社区，其节点变成独立节点或加入其他社区）
- 'remove_nodes'            （移除节点，从图中删除某些节点，可能导致社区结构变化）

In [4]:
# 设置核心参数
transformation='fragment'      #指定演化类型为碎片化
TSS=50 # number of snapshots   #总时间步数（Total Snapshots）
TIME_TO_TRANFORM = 30 # time of the transformation: it corresponds to the number of snapshots needed to complete the transformation
                               #完成碎片化所需的时间步数
assert TIME_TO_TRANFORM<TSS    #确保演化时间不超过总时间
SPEED=1/TIME_TO_TRANFORM       #每个时间步的演化速度（渐进式变化），每步完成1/30的碎片化

# select the initial number of nodes in the graph
# 因为remove_nodes/edges操作会减少网络规模，因此初始节点数更大 
if transformation in ['remove_nodes','remove_edges']: 
    n=3000  #如果是删除节点/边，初始图较大
else: 
    n=1000  #其他情况（比如我们现在是碎片化操作）初始节点数设为1000


n_changes=3 # number of communities affected from the morphing transformations
            # 受演化影响的社区数量（现在是3个社区会逐渐碎片化）
mu = .2 # LFR mixing parameter: how much the communities are mixed between 0 and 1
        # You can also set the othe paramaters of the LFR benchmark in the initialization of GraphMovementSimulation
        # LFR基准网络的混合参数 mu=0表示社区完全独立 mu=1表示社区完全混合 这里设置为0.2 表示社区间连接较少

In [5]:
#generate the folder to store the data:
#创建一个目录路径用于存储生成的动态社区演化数据，通过mu和transformation自动分类存储结果，便于后续分析不同参数下的演化结果
directory_path=f'../results/graphs/mu0{int(mu*10)}/{transformation}/'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [6]:
gname=f'{transformation}_mu0{int(mu*10)}'  #生成图的名称模板
i=0                                        #当前图的编号（从0开始)
sim_count=0                                #失败重试计数器
while True:
    # sometimes the generation of the synthetic static graph fails due to the networkx implementation of the LFR static benchmark, so we need to repeat the generation till we find the right seed. 
    seed=i+sim_count                       #动态生成随机种子（避免重复）
    try:
        #尝试生成图并设置演化
        sim = GraphMovementSimulation(n=n, mu=mu, gname=f'{gname}_{i}', seed=seed)
        G = sim.setup_transformation(transformation, n_splits=n_changes, save=True)
        break #成功则退出循环
    except:
        sim_count+=1 #失败则增加重试次数
print(f'Graph #{i} -> Final seed: {seed}')
#以上操作的目的是为了生成要给符合LFR基准的初始静态图，并应用指定的社区演化


# Create the directory to store the graph:
# 以下操作是保存每次成功生成图的随机种子，确保结果可复现
if not os.path.exists(directory_path+ f'G{i:02}/'):
    # If it doesn't exist, create the directory
    os.makedirs(directory_path+ f'G{i:02}/')

# Store seed information
if os.path.exists(directory_path+'seeds.csv'):
    f=open(directory_path+'seeds.csv','a')
else:
    f=open(directory_path+'seeds.csv','w')
    f.write('num_graph,seed\n')
f.write(f'{i},{seed}\n')
f.close()



Graph #0 -> Final seed: 4


In [7]:
#run the transformation and 
#执行动态社区演化并保存真实社区标签
_,GT=run=sim.run( G, N_Graph=i, delta=SPEED, transformation=transformation,timesteps=TSS,start_trans= 0, stop_trans= TSS)
#返回值是字典，包含社区标签的初始状态和最终状态
pd.DataFrame({'y_true_init':GT['y_true_init'],'y_true_final':GT[ 'y_true_final'] }).fillna('')\
                .to_csv(f'../results/graphs/mu0{int(mu*10)}/{transformation}/G{i:02}/GT.csv.gz',sep=',',index=True,index_label='Node_id')
#将字典转换为DataFrame,然后保存到目录中，并压缩



Processing fragment_mu02_0: 100%|██████████| 50/50 [00:16<00:00,  3.02it/s]
